In [ ]:
def udf_connURLsplit(connURL):
    return connURL.split('/')






In [2]:
def conf_MSSQL(credentials, connType):
    serverName = credentials[0]
    userName = credentials[1]
    password = credentials[2]
    databaseName = credentials[3]

    if connType == 'SQLServer':
        # Configure connection properties for SQL Server
        jdbcUrl = f"jdbc:sqlserver://{serverName};database={databaseName}"
        connectionProperties ={
            "user": userName,
            "password": password,
            "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
        }
    elif connType == 'MySQL':
        # Configure connection properties for MySQL
        jdbcUrl = f"jdbc:mysql://{serverName}:3306/{databaseName}?user={userName}&password={password}"
        connectionProperties = {
            "driver": "com.mysql.jdbc.Driver"
        }
    else:
        raise ValueError("Connection type not recognized")

    credentialsProcessed = [serverName, userName, password, databaseName, jdbcUrl, connectionProperties]
    return credentialsProcessed


In [3]:

def udf_copyDataFileToMSSQL(pipelineUUID,conf_Initial,sourceTable,targetTable,srcCredentials,tgtCredentials,srcConnType,tgtConnType,loadType='full'): # change to incremental

    try:
        # print(tgtConnType)
        execStartTimestamp = str(datetime.datetime.now())[:25]
        tableUUID = str(uuid.uuid4())
        tableName = os.path.basename(sourceTable).split('.')[0]
        tableStatus = 'Completed'
        tableMessage = ''
        srcCnt = ''
        tgtCnt = ''

  
        if srcConnType == 'MySQL':
            src = readData(sourceTable=sourceTable, srcCredentials=srcCredentials, srcConnType=srcConnType, loadType='full')
            srcCnt = src.count()
            if len(targetTable) == 0: targetTable = 'stg.'+os.path.basename(sourceTable).split('.')[0]


        elif (srcConnType=='File'):
            src = readData(sourceTable=sourceTable, srcCredentials=srcCredentials, srcConnType=srcConnType, loadType='full')
            srcCnt = src.count()
            if len(targetTable) == 0: targetTable = 'stg.'+os.path.basename(sourceTable).split('.')[0]

        elif (srcConnType=='SQLServer'):
            src = readData(sourceTable=sourceTable, srcCredentials=srcCredentials, srcConnType=srcConnType, loadType='full')
            srcCnt = src.count()
            if len(targetTable) == 0: targetTable='C:\\Users\\sachin.gupta\\Desktop\\Threading Test Folder SRC\\'+sourceTable

            

        else:
            print('srcConnType is not defined')
            
  
        
        if tgtConnType == 'SQLServer':
            
            tgtCnt = writeData(src, targetTable=targetTable, tgtCredentials=tgtCredentials, tgtConnType=tgtConnType)

        elif tgtConnType=='File':
            tgtCnt = writeData(src, targetTable=targetTable, tgtCredentials=tgtCredentials, tgtConnType=tgtConnType)


            
        execEndTimestamp = str(datetime.datetime.now())[:25]
        values =[pipelineUUID, tableUUID, tableName, execStartTimestamp, execEndTimestamp, tableStatus, tableMessage, srcCnt, tgtCnt]
        udf_insertLogs(conf_Initial[0:4],'table',values)    

    
    except Exception as e:
        execEndTimestamp = str(datetime.datetime.now())[:25]
        tableStatus = 'Failed'
        tableMessage = str(e)[:128]
        values =[pipelineUUID, tableUUID, tableName, execStartTimestamp, execEndTimestamp, tableStatus, tableMessage, srcCnt, tgtCnt]
        udf_insertLogs(conf_Initial[0:4],'table',values)

        
        

    return 0

In [4]:


def copy_files_with_threads(functionName,pipelineUUID,conf_Initial,srcCredentials='',tgtCredentials='',srcTablesList='',tgtTablesList='',srcConnType='',tgtConnType=''):# will take full df here as input with cols: plName,srcTable,tgtTable

    threads = []
   
    for srcDataFile in srcTablesList:
        # target_file_name = 'dbo.'+os.path.basename(source_path).split('.')[0]
        # destination_path = os.path.join(destination_folder, file_name)
        tgtTbl = '' # THIS NEEDS TO BE CHANGED AS PER THE TARGET FUNCTIONALITY
        # udf_copyDataFileToMSSQL(pipelineUUID,conf_Initial,sourceTable,targetTable,srcCredentials,tgtCredentials,srcConnType,tgtConnType='tgt',loadType='full')
        thread = Thread(target=functionName, args=(pipelineUUID,conf_Initial,srcDataFile,tgtTbl,srcCredentials,tgtCredentials,srcConnType,tgtConnType))
        # print(tgtConnType)
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()


In [5]:
def copyData(pipelineUUID, conf_Initial, srcCredentials, tgtCredentials, srcTablesList, tgtTablesList, srcConnType, tgtConnType):
    if (srcConnType == 'File' and tgtConnType == 'SQLServer'):
        copy_files_with_threads(functionName=udf_copyDataFileToMSSQL,
                                pipelineUUID=pipelineUUID,
                                conf_Initial=conf_Initial,
                                srcCredentials=srcCredentials,
                                tgtCredentials=tgtCredentials,
                                srcTablesList=srcTablesList,
                                tgtTablesList=tgtTablesList,
                                srcConnType=srcConnType,
                                tgtConnType=tgtConnType)
    elif( srcConnType == 'MySQL' and tgtConnType == 'SQLServer'):
        copy_files_with_threads(functionName=udf_copyDataFileToMSSQL,
                                pipelineUUID=pipelineUUID,
                                conf_Initial=conf_Initial,
                                srcCredentials=srcCredentials,
                                tgtCredentials=tgtCredentials,
                                srcTablesList=srcTablesList,
                                tgtTablesList=tgtTablesList,
                                srcConnType=srcConnType,
                                tgtConnType=tgtConnType)

    elif srcConnType == 'SQLServer' and tgtConnType == 'File':
        copy_files_with_threads(functionName=udf_copyDataFileToMSSQL,
                                pipelineUUID=pipelineUUID,
                                conf_Initial=conf_Initial,
                                srcCredentials=srcCredentials,
                                tgtCredentials=tgtCredentials,
                                srcTablesList=srcTablesList,
                                tgtTablesList=tgtTablesList,
                                srcConnType=srcConnType,
                                tgtConnType=tgtConnType)
        
    

    else:
        print('Src-Tgt connection combination not recognized')



  

In [6]:
def readData(sourceTable='', srcCredentials='', srcConnType='', loadType='full'):


    
    spark = SparkSession.builder \
        .appName("CSV to SQL Server") \
        .config("spark.driver.extraClassPath", path_sqljdbc_jar).getOrCreate()

    if srcConnType == 'File':
        df = spark.read.csv(sourceTable, header=True, inferSchema=True)
    elif srcConnType == 'SQLServer':
        df = spark.read \
            .format("jdbc") \
            .option("url", srcCredentials[4]) \
            .option("dbtable", sourceTable) \
            .option("user", srcCredentials[1]) \
            .option("password", srcCredentials[2]) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()
    elif srcConnType == 'MySQL':
        path_MYsqljdbc_jar = "C:\\Users\\sachin.gupta\\Downloads\\mysql-connector-j-8.1.0.jar"

        
        spark = SparkSession.builder \
            .appName("CSV to SQL Server") \
            .config("spark.driver.extraClassPath", path_MYsqljdbc_jar).getOrCreate()

        jdbc_url=f"jdbc:mysql://{serverName}/{databaseName}"
        df = spark.read \
            .format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", sourceTable) \
            .option("user", userName) \
            .option("password", password) \
            .option("driver", "com.mysql.jdbc.Driver") \
            .load()

   
    else:
        raise ValueError("Source Connection Type not recognized")

    
    return df


In [7]:
def writeData(df,targetTable,tgtCredentials,tgtConnType):

    if tgtConnType=='SQLServer':
        
        tgtServerName, tgtUserName, tgtPassword, tgtDatabaseName, tgtJdbcUrl, tgtConnectionProperties = tgtCredentials

        spark = SparkSession.builder \
            .appName("CSV to SQL Server") \
            .config("spark.driver.extraClassPath", path_sqljdbc_jar).getOrCreate()

        # schema_query=f"Select column_name from information_schema.columns where Table_Name = %s"
        # params = (TableName)
        # cursor.execute(schema_query,params)
        
        schemaName,tableName = targetTable.split('.')
       
        
        metadata_df = spark.read \
        .format("jdbc") \
        .option("url", tgtJdbcUrl) \
        .option("dbtable", f"(SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{tableName}') AS t") \
        .option("user", tgtUserName) \
        .option("password", tgtPassword) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()
       
       
        tableName='['+tableName+']'

        table_exists = metadata_df.count() > 0
       
        # table_exists = spark.catalog.tableExists(TableName)
        if table_exists:
            df.write.jdbc(url=tgtJdbcUrl, table=schemaName+'.'+tableName, mode="append", properties=tgtConnectionProperties)
        else:
            df.write.jdbc(url=tgtJdbcUrl, table=schemaName+'.'+tableName, mode="overwrite", properties=tgtConnectionProperties)

        tgtCnt = df.count()

    elif tgtConnType=='File':
        df.write \
        .format("csv") \
        .option("header", "true") \
        .mode("overwrite") \
        .save(targetTable)

        
        # spark.stop()
        
    return tgtCnt

In [8]:
def confInitial():

    ServerName = 'psslsql-1.database.windows.net'
    UserName = 'sqladmin'
    Password = 'X*33aMoff92m'
    DatabaseName = 'demodb'

    path_sqljdbc_jar = "C:\\Users\\sachin.gupta\\Desktop\\sqljdbc42-6.0.8112.jar"

    spark = SparkSession.builder \
        .appName("CSV to SQL Server") \
        .config("spark.driver.extraClassPath", path_sqljdbc_jar).getOrCreate()

    jdbc_url = f"jdbc:sqlserver://{ServerName};database={DatabaseName}"

    pipelines_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "cfg.pipelines") \
    .option("user", UserName) \
    .option("password", Password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

    connections_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", f"(SELECT \
        pipelineKey, \
        src.connType as srcConnType,src.connURL as srcConnURL, \
        tgt.connType as tgtConnType,tgt.connURL as tgtConnURL  \
        FROM cfg.pipelines p \
        JOIN cfg.connections src ON srcConn=src.connKey  \
        JOIN cfg.connections tgt ON tgtConn=tgt.connKey) as tbl") \
    .option("user", UserName) \
    .option("password", Password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

    initialConf = [ServerName,UserName,Password,DatabaseName,path_sqljdbc_jar,spark,jdbc_url,pipelines_df,connections_df]

    return initialConf

In [9]:
def connReader(connType='', connURL='', tables_df='', connReaderType=''):
    credentials = ''
    tablesList = ''

    if connReaderType == 'src':
        if connType == 'File':
            # Add srcConnURL to tables_df.tblName
            tables_df = tables_df.withColumn('fullFilePath', sf.concat(lit(connURL), lit('\\'), col("tableName")))
            tablesList = list(tables_df.select(tables_df.fullFilePath).toPandas()['fullFilePath'])
            srcConnDetails = [credentials, tablesList]
            return srcConnDetails
        elif connType == 'SQLServer':
            credentialsList = udf_connURLsplit(connURL)
            credentials = conf_MSSQL(credentialsList, connType)
            tablesList = list(tables_df.select(tables_df.tableName).toPandas()['tableName'])
            srcConnDetails = [credentials, tablesList]
            return srcConnDetails
        elif connType == 'MySQL':
            # For MySQL source connection, credentials will be same as srcConnURL
            credentialsList = udf_connURLsplit(connURL)
            credentials = conf_MSSQL(credentialsList, connType)
            tablesList = list(tables_df.select(tables_df.tableName).toPandas()['tableName'])
            srcConnDetails = [credentials, tablesList]
            return srcConnDetails
        else:
            print('Source Connection Details incorrect or not working')
            return None
    elif connReaderType == 'tgt':
        if connType == 'SQLServer':
            credentialsList = udf_connURLsplit(connURL)
            credentials = conf_MSSQL(credentialsList, connType)
            tgtConnDetails = [credentials, tablesList]
            return tgtConnDetails
        elif connType== 'File':
            tables_df = tables_df.withColumn('fullFilePath', sf.concat(lit(connURL), lit('\\'), col("tableName")))
            tablesList = list(tables_df.select(tables_df.fullFilePath).toPandas()['fullFilePath'])
            tgtConnDetails = [credentials, tablesList]
            return tgtConnDetails
        else:
            print('Target Connection Details incorrect or not working')
            return None
    else:
        print('Connection Type is Invalid')
        return None


In [10]:
def udf_insertLogs(conf_Initial, type, values):

   
    ServerName,UserName,Password,DatabaseName = conf_Initial
    conn = pymssql.connect(server=ServerName, user=UserName, password=Password, database=DatabaseName)
    cursor = conn.cursor()

    if type=='pipeline':
        tableName = 'logs.exec_pipelines'
    if type=='table':
        tableName = 'logs.exec_tables'
    if type=='error':
        tableName = 'logs.exec_errors'

    # var_string = ', '.join('?' * len(values))
    
    insertLogQuery = f"INSERT INTO {tableName} VALUES  %r"% (tuple(values),)
 
    # json.dumps
    cursor.execute(insertLogQuery)

    conn.commit()
    cursor.close()
    conn.close()

    return 0

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit
from pyspark.sql import functions as sf
import uuid

from pyspark.sql.functions import col, when
import datetime

import os
import pymssql

import shutil
import threading
import time
from time import sleep
from threading import Thread
from pyspark.sql.functions import col,lit
from pyspark.sql import functions as sf


conf_Initial = confInitial()
ServerName,UserName,Password,DatabaseName,path_sqljdbc_jar,spark,jdbc_url,pipelines_df,connections_df = conf_Initial

for pipelineKey in list(pipelines_df.select(pipelines_df.pipelineKey).toPandas()['pipelineKey']):
    
    execStartTimestamp = str(datetime.datetime.now())[:25]
    pipelineDetails = pipelines_df.where(col("pipelineKey")==pipelineKey)
    pipelineUUID = str(uuid.uuid4())
    pipelineName = list(pipelineDetails.select(pipelineDetails.pipelineName).toPandas()['pipelineName'])[0]
    pipelineStatus = 'Succeeded'
    pipelineMessage = ''
    
    try:

        tables_df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "cfg.tables") \
        .option("user", UserName) \
        .option("password", Password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()

        tables_df = tables_df.where(col("pipelineKey")==pipelineKey).where(col("loadType")=='full')
        # .where(col("tblName")=='Dimensions.csv')
        # THIS ("loadType")=='full' PART needs to be removed after incremental code addition
        conn = connections_df.where(col("pipelineKey")==pipelineKey)
        
        srcConnType = list(conn.select(conn.srcConnType).toPandas()['srcConnType'])[0]
        srcConnURL = list(conn.select(conn.srcConnURL).toPandas()['srcConnURL'])[0]
        tgtConnType = list(conn.select(conn.tgtConnType).toPandas()['tgtConnType'])[0]
        tgtConnURL = list(conn.select(conn.tgtConnURL).toPandas()['tgtConnURL'])[0]

        srcConnDetails = connReader(srcConnType, srcConnURL, tables_df, 'src')
        tgtConnDetails = connReader(tgtConnType, tgtConnURL, tables_df, 'tgt')
        # srcConnDetails can be--> filesList,credentailsList
        # tgtConnDetails can be--> only credentailsList

        srcCredentials, srcTablesList = srcConnDetails
        tgtCredentials, tgtTablesList = tgtConnDetails

   
        copyData(pipelineUUID,conf_Initial,srcCredentials,tgtCredentials,srcTablesList,tgtTablesList,srcConnType,tgtConnType)

        exec_tables = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "[logs].[exec_tables]") \
        .option("user", UserName) \
        .option("password", Password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()

        exec_tables = exec_tables.where(col("pipelineUUID")==pipelineUUID)
       
        

        has_failed = exec_tables.filter(col('tableStatus') == 'Failed').count() > 0

        pipelineStatus = 'Failed' if has_failed else pipelineStatus

        if pipelineStatus == 'Failed':
                # Raise a custom exception to force the code to enter the except block
            raise Exception("Pipeline Failed")
        
       
        execEndTimestamp = str(datetime.datetime.now())[:25]
        values = [pipelineUUID,	pipelineName, execStartTimestamp, execEndTimestamp,	pipelineStatus,	pipelineMessage]
      
        udf_insertLogs(conf_Initial[0:4],'pipeline',values)
        


    except Exception as e:
        execEndTimestamp = str(datetime.datetime.now())[:25]
        pipelineStatus = 'Failed'
        pipelineMessage = str(e)
        values = [pipelineUUID, pipelineName, execStartTimestamp, execEndTimestamp, pipelineStatus, pipelineMessage]
        udf_insertLogs(conf_Initial[0:4],'pipeline',values)
        spark.stop()


    # def copyData(srcConnType,srcConnDetails,tgtConnType,tgtConnDetails):

    #     if(srcConnType=='File' and tgtConnType=='SQLServer'):
    #         # tgtCredentials = conf_MSSQL(tgtConnDetails)
    #         copy_files_with_threads(srcList=srcConnDetails,srcConnType=srcConnType,functionName=udf_copyDataFileToMSSQL,tgtCredentials=tgtConnDetails,tgtConnType=tgtConnType)
    #     else:
    #         print('Src-Tgt connection type not recognised')

    # ,srcConnType,srcConnDetails,tgtConnType,tgtConnDetails
    # copy_files_with_threads(functionName=copyData,srcCredentials=srcCredentials,tgtCredentials=tgtCredentials,
    #                         srcTablesList=srcTablesList,tgtTablesList=tgtTablesList,srcConnType=srcConnType,tgtConnType=tgtConnType)
    # copyData(srcConnType,srcConnDetails,tgtConnType,tgtConnDetails)
            